In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime

In [ ]:
DOWNLOAD_FOLDER_NAME = "downloads"

In [ ]:
def get_file_info(file_path):
    with open(file_path, 'r') as f:
        data = f.read()
        soup = BeautifulSoup(data, "lxml")

        datum = get_datum(soup)
        print(f"Date: {datum}")

        parteien = list(set([str(partei.text).replace(u'\xa0',u' ') for partei in soup.find_all("p", klasse="AL_Partei")]))
        print(f"Parteien: {len(parteien)}\n{parteien}")

        wahlen = get_wahlen(soup)
        print(f"Durchgeführte Wahlen: {len(wahlen)}")

        entschuldigte = get_entschuldigte(soup)
        print(f"Entschuldigte Abgeordnete: {len(entschuldigte)}\n{entschuldigte}")

        mutterschutz_entschuldigte = get_mutterschutz_entschuldigte(soup)
        print(f"davon im Mutterschutz: {len(mutterschutz_entschuldigte)}\n{mutterschutz_entschuldigte}")

        redner = get_redner(soup)
        print(f"Redner: {len(redner)}\n{redner}")



In [ ]:
def get_df():
    datum_list = []

    entschuldigte_list = []
    davon_mutterschutz_list = []

    wahlen_list = []

    for folder in os.listdir(DOWNLOAD_FOLDER_NAME):
        for file in os.listdir(Path(DOWNLOAD_FOLDER_NAME, folder)):
            if file.endswith(".xml"):
                print(f"Auswertung von {file}")
                with open(Path(DOWNLOAD_FOLDER_NAME, folder, file), 'r') as f:
                    data = f.read()
                    soup = BeautifulSoup(data, "lxml")

                    datum = datetime.strptime(get_datum(soup), "%d.%m.%Y")
                    print(f"Datum: {datum}")
                    datum_list.append(datum)


                    entschuldigte = get_entschuldigte(soup)
                    print(f"Entschuldigte Abgeordnete: {len(entschuldigte)}\n{entschuldigte}")
                    entschuldigte_list.append(len(entschuldigte))

                    mutterschutz_entschuldigte = get_mutterschutz_entschuldigte(soup)
                    print(f"davon im Mutterschutz: {len(mutterschutz_entschuldigte)}\n{mutterschutz_entschuldigte}")
                    davon_mutterschutz_list.append(len(mutterschutz_entschuldigte))

                    anzahl_wahlen = get_anzahl_wahlen(soup)
                    print(f"Durchgeführte Wahlen: {anzahl_wahlen}")
                    wahlen_list.append(anzahl_wahlen)

    
    df = pd.DataFrame(
        {
            "entschuldigte":entschuldigte_list, 
            "davon_mutterschutz":davon_mutterschutz_list,
            "wahlen":wahlen_list
        },
        index=pd.to_datetime(datum_list)
    )
    df.sort_index(inplace=True)
    return df

In [ ]:
def get_anzahl_wahlen(soup):
    wahlen = soup.select('anlagen-text[anlagen-typ="Ergebnis und Namensverzeichnis"]')
    for wahl in wahlen:
        if wahl.find("p", klasse="T_fett") is not None:
            wahltitel = wahl.find("p", klasse="T_fett").text.replace(u'\xa0',u' ').replace("der Mitglieder des Deutschen Bundestages, die an der ","").replace("teilgenommen haben","")
        else:
            wahltitel = wahl.find("p", klasse="Anlage_3").text.replace(u'\xa0',u' ').replace("der Mitglieder des Deutschen Bundestages, die an der ","").replace("teilgenommen haben","")
        
            
        
        if wahl.table is not None and wahl.table.caption is not None:
            abgegebene_stimmen = wahl.table.caption.text.replace(u'\xa0',u' ').replace("Abgegebene Stimmkarten: ","").replace(" Ergebnis","")
        else:
            abgegebene_stimmen = wahl.find(lambda tag:tag.name=="p" and "Abgegebene Stimmkarten" in tag.text.replace(u'\xa0',u' ')).text.replace(u'\xa0',u' ').replace("Abgegebene Stimmkarten: ","").replace(" Ergebnis","")
        
        if " Für die Wahl sind mindestens" in abgegebene_stimmen:
            abgegebene_stimmen = abgegebene_stimmen.split(" Für die Wahl sind mindestens")[0]
        abgegebene_stimmen = int(abgegebene_stimmen)
        print(f"Wahl: {wahltitel}, Abgegebene Stimmen: {abgegebene_stimmen}")
    return len(wahlen)

In [ ]:
def get_datum(soup):
    date = soup.find_all("datum")[0]["date"]
    return date

In [ ]:
def get_redner(soup):
    rednerliste_div = soup.find("rednerliste")
    redner = [
        get_normale_namen(redner.find("titel").text + " " + redner.find("vorname").text + " " +  redner.find("nachname").text)
        if redner.find("titel") is not None 
        else get_normale_namen(redner.find("vorname").text + " " +  redner.find("nachname").text)
        for redner in rednerliste_div.find_all("redner")]
    return redner

In [ ]:
def get_entschuldigte(soup):
    entschuldigte_div = soup.select_one('anlagen-text[anlagen-typ="Entschuldigte Abgeordnete"]')
    if entschuldigte_div is None:
        entschuldigte_div = soup.select_one('anlagen-text[anlagen-typ="Liste der entschuldigten Abgeordneten"]')
        if entschuldigte_div is None:
            print("Keine entschuldigten Abgeordneten gefunden")
            return []
    if entschuldigte_div.tbody is None:
        print("Keine entschuldigten Abgeordneten gefunden")
        return []
    entschuldigte = [
        get_normale_namen(tr.td.text) for tr in entschuldigte_div.tbody.find_all("tr")
        if tr.td is not None
    ]
    return entschuldigte

In [ ]:
def get_mutterschutz_entschuldigte(soup):
    entschuldigte_div = soup.select_one('anlagen-text[anlagen-typ="Entschuldigte Abgeordnete"]')
    if entschuldigte_div is None:
        entschuldigte_div = soup.select_one('anlagen-text[anlagen-typ="Liste der entschuldigten Abgeordneten"]')
        if entschuldigte_div is None:
            print("Keine entschuldigten Abgeordneten gefunden")
            return []
    if entschuldigte_div.tbody is None:
        print("Keine entschuldigten Abgeordneten gefunden")
        return []
    mutterschutz_entschuldigte = [
        get_normale_namen(tr.td.text) for tr in entschuldigte_div.tbody.find_all("tr") 
        if tr.td is not None
        and ("*" in tr.td.text or "(aufgrund gesetzlichen Mutterschutzes)" in tr.td.text)
    ]
    return mutterschutz_entschuldigte

In [ ]:
def get_normale_namen(name):
    name = name.replace("*","").replace("(aufgrund gesetzlichen Mutterschutzes)","").replace(u'\xa0',u' ')
    if "," in name:
        split_name = name.split(",")
        name = split_name[1] + " " + split_name[0]
    name = name.replace("  "," ")
    if name[0] == " ":
        name = name[1:]
    if name[-1] == " ":
        name = name[:-1]
    return name